In [ ]:
# Imports
import os
import numpy as np
import glob
import pandas as pd
from mat4py import loadmat
import ipdb
import math 

from gaze_utils import *


# figure imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.express as px
from plot_utils import plotly_template

In [ ]:
# Define parameters
num_trials = 101                       # number of trials per run
#parameters for model (saccade detection)
sampling_rate = 1000                   # eyetrakcing sampling rate
velocity_th = 1.5                      # velocity sd threshold
min_dur = 20                           # threshold minimum duration
merge_interval = 20                    # interval between saccade events
cor_sac_onset_th = 100                 # corrective saccade onset threshold (inferior or equal in ms)
fix_trial_end_soa = 800                # duration between fixation offset and trial end for blink checking 
fix_area_rad = 2                       # saccade onset position tolerance area in dva
sac_area_rad = 3                       # saccade landing area tolerance in dva
sac_lat_min = 50                       # saccade latency minimum duration
sac_lat_max = 400                      # saccade latency maximum duration

In [ ]:
# Define folders
# when on mac
base_dir = r'/Users/sinakling/projects/deepmreyeclosed' 
data_dir = '{}/data'.format(base_dir)
subject = 'sub-01'

subject_num = subject[4:]
fig_dir = '{data_dir}/{subject}/figures'.format(data_dir=data_dir, subject=subject)
training = 0

In [ ]:
# Define data filenames
if training == 1:
    data_events = sorted(glob.glob(r'{}/{}/ses-01/beh/*.tsv'.format(data_dir,subject)))
    num_run = len(data_events)

    data_eyetrack = sorted(glob.glob(r'{}/{}/ses-01/beh/*.edf'.format(data_dir,subject)))
    data_mat = sorted(glob.glob(r'{}/{}/ses-01/beh/*.mat'.format(data_dir,subject)))
else: 
    data_events = sorted(glob.glob(r'{}/{}/ses-01/func/*.tsv'.format(data_dir,subject)))
    num_run = len(data_events)

    data_eyetrack = sorted(glob.glob(r'{}/{}/ses-01/func/*.edf'.format(data_dir,subject)))
    data_mat = sorted(glob.glob(r'{}/{}/ses-01/func/*.mat'.format(data_dir,subject)))



assert len(data_eyetrack) > 0, "No eyetracking data found"
assert len(data_mat) > 0, "No matlab data found"
assert len(data_events) > 0, "No event files found"

In [ ]:
# for sub 02 with missing run 01

del data_events[0]
del data_mat[0]
del data_eyetrack[0]

In [ ]:
# Create message and data files (run only once first time)
for run in data_eyetrack:
    
    if not os.path.exists(run.replace('.edf','.msg')):
        #os.system('edf2asc.exe {} -e -y'.format(run))    #! windows EXECUTE through cmd (start edf2asc "/Users/sinakling/projects/deepmreyeclosed/data/sub-01/ses-01/beh/sub-01_ses-01_task-DeepMReyeClosed_training_run-01_eyetrack.edf" -e -y)
        os.rename(run.replace('.edf','.asc'),run.replace('.edf','.msg'))    

In [ ]:
# Create dat files (run only once first time)
for run in data_eyetrack: 
    if not os.path.exists(run.replace('.edf','.dat')): 
       # os.system('edf2asc.exe {} -s -miss -1.0 -y'.format(run)) #!EXECUTE through cmd  (start edf2asc "/Users/sinakling/projects/deepmreyeclosed/data/sub-01/ses-01/beh/sub-01_ses-01_task-DeepMReyeClosed_training_run-01_eyetrack.edf" -s -miss -1.0 -y)
       os.rename(run.replace('.edf','.asc'),run.replace('.edf','.dat'))

In [ ]:
# Collect MSG data
# saves timestamps from messages send to eyetracking data in dataframe: one per run, one for all runs together per subject


#adapted for DeepMReyeCalib 

msg_outputs = ['trial_onset', 'trial_offset']
 
for msg_output in msg_outputs:
    exec("{} = np.zeros(num_trials*num_run)".format(msg_output))

t_run = 0
dfs_list = []

record_lines = []

for run in data_eyetrack:
    
    msgfid = open(run.replace('.edf','.msg'))
    first_last_time, first_time, last_time = False, False, False

    while not first_last_time:
        line_read = msgfid.readline()

        if not line_read == '':
            la = line_read.split()
    
            if len(la) > 2:
                if la[2] == 'RECORD_START' and not first_time:                      #Eyelink('message', 'RECORD_START'); 
                    first_time = True
                    record_lines.append(line_read)
                if la[2] == 'RECORD_STOP' and not last_time:                        #Eyelink('message', 'RECORD_STOP');
                    last_time = True
                    record_lines.append(line_read)
            if len(la) > 4:
                if la[2] == 'trial' and la[4]=='started':                           #Eyelink('message', '%s', sprintf('trial %i started\n', t));
                    trial_onset[int(la[3])-1 + (t_run)*num_trials] = float(la[1])   
                if la[2] == 'trial' and la[4]=='ended':                             #Eyelink('message', '%s', sprintf('trial %i ended\n', t));
                    trial_offset[int(la[3])-1 + (t_run)*num_trials] = float(la[1])
                
    
        if first_time and last_time:
            first_last_time = True
            msgfid.close();
    t_run += 1

# create events dataframe
for run_num, run in enumerate(data_events):
    df_run = pd.read_csv(run, sep="\t")
    if run_num  > 0 :
        df_events = pd.concat([df_events, df_run])
    else :
        df_events = df_run
        

msg_dict = {}
for msg_output in msg_outputs:
    eval("msg_dict.update({'%s':%s})"%(msg_output,msg_output))
    

df_msg = pd.DataFrame(msg_dict)
df_all = pd.concat([df_events.reset_index(drop=True),
                    df_msg.reset_index(drop=True)], axis=1)

# Save DataFrame for each run
num_trials_per_run = int(len(df_all) / num_run)
for i in range(num_run):
    start_idx = i * num_trials_per_run
    end_idx = (i + 1) * num_trials_per_run if i < (num_run - 1) else len(df_all)
    df_run_single = df_all.iloc[start_idx:end_idx]
    df_run_single.to_csv(f'{subject}_closed_run_{i + 1}_messages.csv', index=False)

# Append DataFrame to the list
dfs_list.append(df_all)

t_run += 1

# Concatenate all DataFrames from the list into a final DataFrame
final_df = pd.concat(dfs_list, ignore_index=True)

# Save the final concatenated DataFrame
final_df.to_csv(f'{subject}_closed_all_runs_messages.csv', index=False)


In [ ]:
# Extract recording times from msg file 

record_dict = {}
record_start_count = 1
record_stop_count = 1

for line in record_lines:
    parts = line.split()
    timestamp = int(parts[1])
    action = parts[2]

    if action == 'RECORD_START':
        key = f'RECORD_START_{record_start_count}'
        record_dict[key] = timestamp
        record_start_count += 1
    elif action == 'RECORD_STOP':
        key = f'RECORD_STOP_{record_stop_count}'
        record_dict[key] = timestamp
        record_stop_count += 1

print(record_dict)

In [ ]:
# Read message csvs
df_run_1 =  pd.read_csv(f'{subject}_closed_run_1_messages.csv')
df_run_2 =  pd.read_csv(f'{subject}_closed_run_2_messages.csv')
df_run_3 =  pd.read_csv(f'{subject}_closed_run_3_messages.csv')

dfs_runs = [df_run_1,df_run_2, df_run_3]

# for sub 02 with missing run 01

dfs_runs = [df_run_1,df_run_2,df_run_3]

df_run_1

In [ ]:
print((df_run_1['trial_onset'][100]-df_run_1['trial_onset'][0])/60)

In [ ]:
import scipy.io

matfile = scipy.io.loadmat(data_mat[0])

iti_dur = matfile['config']['const'][0,0]['iti_dur_sec'][0][0][0][0]
trial_dur = matfile['config']['const'][0,0]['trial_dur_sec'][0][0][0][0]

durations_mat = [trial_dur] * 101

durations_mat[0] = iti_dur
durations_mat[25] = iti_dur
durations_mat[50] = iti_dur
durations_mat[75] = iti_dur
durations_mat[100] = iti_dur


In [ ]:
df_run_1['duration'] = durations_mat
#df_run_2['duration'] = durations_mat
#df_run_3['duration'] = durations_mat

In [ ]:
# compute time instruction reading time in beginn# compute time instruction reading time in beginning 

# Print timing check statements

instruction_times = []

for run_num,run in zip(range(len(dfs_runs)), dfs_runs):
    duration = int(run['trial_onset'][0]) - record_dict.get(f'RECORD_START_{run_num + 1}')
    instruction_times.append(duration) 
    print(f"Duration of experiment in run {run_num + 1} was {(run['trial_offset'][100] - run['trial_onset'][0])/60000} minutes")
    print(f"Duration of eyelink recording in run {run_num + 1} was {(record_dict.get(f'RECORD_STOP_{run_num + 1}')- record_dict.get(f'RECORD_START_{run_num + 1}'))/60000} minutes")

#add 500ms for plot accuracy
instruction_times = [x + 500 for x in instruction_times]

print(instruction_times)

ending_times = []

for run_num,run in zip(range(len(dfs_runs)), dfs_runs):
    duration = int(run["trial_offset"][100]) - record_dict.get(f'RECORD_STOP_{run_num + 1}')
    ending_times.append(duration) 

print(ending_times)

In [ ]:
# overwrite instruction_times for sub-01

instruction_times[0] = 30000

print(instruction_times)



In [ ]:
# Remove lines with BLINK message

# Define the number of runs
num_runs = 3

for run_num, data in zip(range(1, num_runs + 1), data_eyetrack):
    # Load eye tracking data for each run
    eye_data_run = np.genfromtxt(data.replace('.edf', '.dat'), usecols=(0, 1, 2))

    # Initialize variables for each run
    eye_data_runs = eye_data_run
    blinkNum = 0
    blink_start = False

    for tTime in np.arange(0, eye_data_runs.shape[0], 1):
        if not blink_start:
            if eye_data_runs[tTime, 1] == -1:
                blinkNum += 1
                timeBlinkOnset = eye_data_runs[tTime, 0]
                blink_start = True
                if blinkNum == 1:
                    blink_onset_offset = np.matrix([timeBlinkOnset, np.nan])
                else:
                    blink_onset_offset = np.vstack((blink_onset_offset, [timeBlinkOnset, np.nan]))

        if blink_start:
            if eye_data_runs[tTime, 1] != -1:
                timeBlinkOffset = eye_data_runs[tTime, 0]
                blink_start = 0
                blink_onset_offset[blinkNum - 1, 1] = timeBlinkOffset

    # Nan record around detected blinks
    eye_data_runs_nan_blink = np.copy(eye_data_runs)

    # Extend time range
    time_range_extension = 200

    for tBlink in np.arange(0, blinkNum, 1):
        onset_time = blink_onset_offset[tBlink, 0]
        offset_time = blink_onset_offset[tBlink, 1]

        # Define the extended time range
        extended_onset = onset_time - time_range_extension
        extended_offset = offset_time + time_range_extension

        # Set values in the specified time range to NaN
        eye_data_runs_nan_blink[
            np.logical_and(
                eye_data_runs_nan_blink[:, 0] >= extended_onset,
                eye_data_runs_nan_blink[:, 0] <= extended_offset,
            ),
            1,
        ] = np.nan

        eye_data_runs_nan_blink[
            np.logical_and(
                eye_data_runs_nan_blink[:, 0] >= extended_onset,
                eye_data_runs_nan_blink[:, 0] <= extended_offset,
            ),
            2,
        ] = np.nan

    # Save the modified data for each run
    output_file_path = f"eye_data_{subject}_closed_run_{run_num}_nan_blink.csv"
    np.savetxt(output_file_path, eye_data_runs_nan_blink, delimiter=",")


In [ ]:
import scipy.io

matfile = scipy.io.loadmat(data_mat[0])
scr_sizeX = matfile['config']['scr'][0,0]['scr_sizeX'][0][0][0][0]
scr_sizeY = matfile['config']['scr'][0,0]['scr_sizeY'][0][0][0][0]
screen_size = np.array([scr_sizeX,scr_sizeY])
ppd = matfile['config']['const'][0,0]['ppd'][0][0][0][0]


In [ ]:
# Load eyedata per run as np array
# Center and convert to dva
eye_data_run_1 = pd.read_csv(f"eye_data_{subject}_closed_run_1_nan_blink.csv")
eye_data_run_1 = np.array(eye_data_run_1)

x_dva_run_1 = (eye_data_run_1[:,1] - (screen_size[0]/2))/ppd
y_dva_run_1 =  -1.0*((eye_data_run_1[:,2] - (screen_size[1]/2))/ppd)



eye_data_run_2 = pd.read_csv(f"eye_data_{subject}_closed_run_2_nan_blink.csv")
eye_data_run_2 = np.array(eye_data_run_2)

x_dva_run_2 = (eye_data_run_2[:,1] - (screen_size[0]/2))/ppd
y_dva_run_2 = -1.0*((eye_data_run_2[:,2] - (screen_size[1]/2))/ppd)



eye_data_run_3 = pd.read_csv(f"eye_data_{subject}_closed_run_3_nan_blink.csv")
eye_data_run_3 = np.array(eye_data_run_3)

x_dva_run_3 = (eye_data_run_3[:,1] - (screen_size[0]/2))/ppd
y_dva_run_3 = -1.0*((eye_data_run_3[:,2] - (screen_size[1]/2))/ppd)


eye_data_all_runs = [eye_data_run_1,eye_data_run_2,eye_data_run_2]




In [ ]:
# for sub 02 

eye_data_all_runs = [eye_data_run_2,eye_data_run_2]

In [ ]:
eyetracking_data_all_runs_x = [x_dva_run_1[instruction_times[0]:ending_times[0]],x_dva_run_2[instruction_times[1]:ending_times[1]],x_dva_run_3[instruction_times[2]:ending_times[2]]]
eyetracking_data_all_runs_y = [y_dva_run_1[instruction_times[0]:ending_times[0]],y_dva_run_2[instruction_times[1]:ending_times[1]],y_dva_run_3[instruction_times[2]:ending_times[2]]]

In [ ]:
# for sub 02, with missing run 1

eyetracking_data_all_runs_x = [x_dva_run_2[instruction_times[1]:ending_times[1]],x_dva_run_3[instruction_times[2]:ending_times[2]]]
eyetracking_data_all_runs_y = [y_dva_run_2[instruction_times[1]:ending_times[1]],y_dva_run_3[instruction_times[2]:ending_times[2]]]

In [ ]:
# Interpolate blink periods for plots 

def interpol_nans(eyetracking_data_list):
    interpolated_data_list = []
    for i in range(len(eyetracking_data_list)):
        eyetracking_no_nans = np.nan_to_num(eyetracking_data_list[i])
        nan_indices = np.isnan(eyetracking_data_list[i])

        # Interpolate NaN values using linear interpolation
        eyetracking_signal_interpolated = np.interp(np.arange(len(eyetracking_data_list[i])), np.where(~nan_indices)[0], eyetracking_no_nans[~nan_indices])
        interpolated_data_list.append(eyetracking_signal_interpolated)

    return interpolated_data_list


In [ ]:
# extract fixation point position 
triangle_positions = [
    [-9.0, 9.0], [9.0, 9.0],
            [0.0, 0.0],
    [-9.0, -9.0],[9.0, -9.0]
]



all_runs_expected_x = []
all_runs_expected_y = []

trial_durations_all_runs = []

for df_run, eye_data in zip(dfs_runs,eye_data_all_runs): 
    expected_position = []

    for i in range(0,num_trials): 
        if not pd.isna(df_run['fixation_position'][i]):
            index = int(df_run['fixation_position'][i])-1
            expected_position.append(triangle_positions[index])

        else: 
            expected_position.append(np.nan)

    expected_position_x = [item[0] if isinstance(item, list) else item for item in expected_position]
    
    expected_position_y = [item[1] if isinstance(item, list) else item for item in expected_position]
    


    # check expected fixation positions 
    fig = px.scatter(x=expected_position_x, y=expected_position_y, width=800, height=800, template= "simple_white")
    fig.update_xaxes(range=[-10,10])
    fig.update_yaxes(range=[-10,10])

    #fig.show()


    # Align trial based expected position with milisecond format of eyetracking data

    trial_durations = df_run['duration']*1000
    run_durations = np.cumsum(trial_durations)
    trial_durations_all_runs.append(list(run_durations))

    # X direction
    trial_indices_x = np.searchsorted(run_durations, np.arange(len(eye_data[:,0]))).astype(int)
    
    expected_target_positions_x = np.array(expected_position_x)
    trial_indices_x = np.clip(trial_indices_x, 0, len(expected_target_positions_x) - 1)
    expected_position_x_aligned = expected_target_positions_x[trial_indices_x]
    all_runs_expected_x.append(expected_position_x_aligned)

    # Y direction
    trial_indices_y = np.searchsorted(run_durations, np.arange(len(eye_data[:,0]))).astype(int)
    
    expected_target_positions_y = np.array(expected_position_y)
    trial_indices_y = np.clip(trial_indices_y, 0, len(expected_target_positions_y) - 1)
    expected_position_y_aligned = expected_target_positions_y[trial_indices_y]
    all_runs_expected_y.append(expected_position_y_aligned)

In [ ]:
np.save("trial_duratations", trial_durations_all_runs[0])

In [ ]:
eyetracking_data_all_runs_x_task_1 = [eyetracking_data_all_runs_x[0][int(trial_durations_all_runs[0][0]):int(trial_durations_all_runs[0][23])],eyetracking_data_all_runs_x[1][int(trial_durations_all_runs[1][0]):int(trial_durations_all_runs[1][23])],eyetracking_data_all_runs_x[2][int(trial_durations_all_runs[2][0]):int(trial_durations_all_runs[2][23])]]
eyetracking_data_all_runs_x_task_3 = [eyetracking_data_all_runs_x[0][int(trial_durations_all_runs[0][50]):int(trial_durations_all_runs[0][73])],eyetracking_data_all_runs_x[1][int(trial_durations_all_runs[1][50]):int(trial_durations_all_runs[1][73])],eyetracking_data_all_runs_x[2][int(trial_durations_all_runs[2][50]):int(trial_durations_all_runs[2][73])]]


eyetracking_data_all_runs_y_task_1 = [eyetracking_data_all_runs_y[0][int(trial_durations_all_runs[0][0]):int(trial_durations_all_runs[0][23])],eyetracking_data_all_runs_y[1][int(trial_durations_all_runs[1][0]):int(trial_durations_all_runs[1][23])],eyetracking_data_all_runs_y[2][int(trial_durations_all_runs[2][0]):int(trial_durations_all_runs[2][23])]]
eyetracking_data_all_runs_y_task_3 = [eyetracking_data_all_runs_y[0][int(trial_durations_all_runs[0][50]):int(trial_durations_all_runs[0][73])],eyetracking_data_all_runs_y[1][int(trial_durations_all_runs[1][50]):int(trial_durations_all_runs[1][73])],eyetracking_data_all_runs_y[2][int(trial_durations_all_runs[2][50]):int(trial_durations_all_runs[2][73])]]



In [ ]:
# for sub 02 : missing run 01 

eyetracking_data_all_runs_x_task_1 = [eyetracking_data_all_runs_x[1][int(trial_durations_all_runs[1][0]):int(trial_durations_all_runs[1][23])],eyetracking_data_all_runs_x[2][int(trial_durations_all_runs[2][0]):int(trial_durations_all_runs[2][23])]]
eyetracking_data_all_runs_x_task_3 = [eyetracking_data_all_runs_x[1][int(trial_durations_all_runs[1][50]):int(trial_durations_all_runs[1][73])],eyetracking_data_all_runs_x[2][int(trial_durations_all_runs[2][50]):int(trial_durations_all_runs[2][73])]]


eyetracking_data_all_runs_y_task_1 = [eyetracking_data_all_runs_y[1][int(trial_durations_all_runs[1][0]):int(trial_durations_all_runs[1][23])],eyetracking_data_all_runs_y[2][int(trial_durations_all_runs[2][0]):int(trial_durations_all_runs[2][23])]]
eyetracking_data_all_runs_y_task_3 = [eyetracking_data_all_runs_y[1][int(trial_durations_all_runs[1][50]):int(trial_durations_all_runs[1][73])],eyetracking_data_all_runs_y[2][int(trial_durations_all_runs[2][50]):int(trial_durations_all_runs[2][73])]]



In [ ]:
eyetracking_data_all_runs_x_task_1_interpol = interpol_nans(eyetracking_data_all_runs_x_task_1)
eyetracking_data_all_runs_x_task_3_interpol = interpol_nans(eyetracking_data_all_runs_x_task_3)

eyetracking_data_all_runs_y_task_1_interpol = interpol_nans(eyetracking_data_all_runs_y_task_1)
eyetracking_data_all_runs_y_task_3_interpol = interpol_nans(eyetracking_data_all_runs_y_task_3)

In [ ]:
for i in range(len(eyetracking_data_all_runs_x)):

    eyetracking_data_all_runs_x[i][int(trial_durations_all_runs[i][0]):int(trial_durations_all_runs[i][23])] = eyetracking_data_all_runs_x_task_1_interpol[i] 
    eyetracking_data_all_runs_x[i][int(trial_durations_all_runs[i][50]):int(trial_durations_all_runs[i][73])] = eyetracking_data_all_runs_x_task_3_interpol[i] 

    eyetracking_data_all_runs_y[i][int(trial_durations_all_runs[i][0]):int(trial_durations_all_runs[i][23])] = eyetracking_data_all_runs_y_task_1_interpol[i] 
    eyetracking_data_all_runs_y[i][int(trial_durations_all_runs[i][50]):int(trial_durations_all_runs[i][73])] = eyetracking_data_all_runs_y_task_3_interpol[i] 

In [ ]:
np.save(f"eyetracking_x_data_Closed_{subject}_run_1", eyetracking_data_all_runs_x[0])
np.save(f"eyetracking_y_data_Closed_{subject}_run_1", eyetracking_data_all_runs_y[0])

np.save(f"eyetracking_x_data_Closed_{subject}_run_2", eyetracking_data_all_runs_x[1])
np.save(f"eyetracking_y_data_Closed_{subject}_run_2", eyetracking_data_all_runs_y[1])

np.save(f"eyetracking_x_data_Closed_{subject}_run_3", eyetracking_data_all_runs_x[2])
np.save(f"eyetracking_y_data_Closed_{subject}_run_3", eyetracking_data_all_runs_y[2])

In [ ]:


np.save(f"expected_x_data_Closed_{subject}_run_1", all_runs_expected_x[0])
np.save(f"expected_y_data_Closed_{subject}_run_1", all_runs_expected_y[0])

np.save(f"expected_x_data_Closed_{subject}_run_2", all_runs_expected_x[1])
np.save(f"expected_y_data_Closed_{subject}_run_2", all_runs_expected_y[1])

np.save(f"expected_x_data_Closed_{subject}_run_3", all_runs_expected_x[2])
np.save(f"expected_y_data_Closed_{subject}_run_3", all_runs_expected_y[2])

In [ ]:
deepmreye_sub_01_run_1_X = np.load("/Users/sinakling/projects/DeepMReye/closed_data/closed_deepmreye_X_sub-01_run_01.npy")
deepmreye_sub_01_run_2_X = np.load("/Users/sinakling/projects/DeepMReye/closed_data/closed_deepmreye_X_sub-01_run_02.npy")
deepmreye_sub_01_run_3_X = np.load("/Users/sinakling/projects/DeepMReye/closed_data/closed_deepmreye_X_sub-01_run_03.npy")

deepmreye_data_all_runs_x = [deepmreye_sub_01_run_1_X,deepmreye_sub_01_run_2_X,deepmreye_sub_01_run_3_X]

deepmreye_sub_01_run_1_Y= np.load("/Users/sinakling/projects/DeepMReye/closed_data/closed_deepmreye_Y_sub-01_run_01.npy")
deepmreye_sub_01_run_2_Y = np.load("/Users/sinakling/projects/DeepMReye/closed_data/closed_deepmreye_Y_sub-01_run_02.npy")
deepmreye_sub_01_run_3_Y = np.load("/Users/sinakling/projects/DeepMReye/closed_data/closed_deepmreye_Y_sub-01_run_03.npy")

deepmreye_data_all_runs_y = [deepmreye_sub_01_run_1_Y,deepmreye_sub_01_run_2_Y,deepmreye_sub_01_run_3_Y]

In [ ]:
import plotly.graph_objects as gosss

# Sample data
# task 1: :100000 (inkl itis)
# task 2: 91000:192000 (inkl itis)
# task 3: 1883000:25000 (inkl itis)
actual_data = avg_diff_x
expected_data = all_runs_expected_x[0]


# Create a trace for the actual data
trace_actual = go.Scatter(
    x=list(range(1, len(actual_data)+1)),
    y=actual_data,
    showlegend=True,
    mode='lines',
    name='Eyetracking Gaze Position',
    line=dict(color='#CD5334', width=2.5),
)


trace_expected = go.Scatter(
    x=list(range(1, len(actual_data)+1)),
    y=expected_data,
    showlegend=True,
    mode='lines',
    name='Expected Gaze Position',
    line=dict(color='#45B69C', width=2.5),
)

# Create layout with specific style for a nature paper
layout = go.Layout(
    title=f'DeepMReyeClosed {subject}',
    title_x = 0.5,
    xaxis=dict(title='Time (ms)'),
    yaxis=dict(title='Y Gaze Position', range = [-10,10]),
    showlegend=True,
    legend=dict(x=1, y=1),
    font=dict(family='Arial', size=12, color='black'),
    paper_bgcolor='white',  # Background color
    plot_bgcolor='white',   # Plot area background color
    margin=dict(l=50, r=50, t=50, b=50),  # Margins
)

# Create figure
fig = go.Figure(data=[trace_actual], layout=layout)


fig.add_vrect(x0="7000", x1="91000", 
            label=dict(
            text="eyes open",
            textposition="top center"),
            fillcolor="#DCDCE5", opacity=0.2, line_width=0)

fig.add_vrect(x0="100000", x1="184000", 
            label=dict(
            text="eyes part. closed",
            textposition="top center"),
            fillcolor="#D8DDEF", opacity=0.2, line_width=0)

fig.add_vrect(x0="193000", x1="276000", 
            label=dict(
            text="no stim.",
            textposition="top center"),
            fillcolor="#A0A4B8", opacity=0.1, line_width=0)

fig.add_vrect(x0="284000", x1="371000", 
            label=dict(
            text="eyes closed",
            textposition="top center"),
            fillcolor="#7293A0", opacity=0.1, line_width=0)

# Show the plot

fig.show()



# × 
import plotly.io as pio
#pio.write_image(fig, f'y_closed_{subject}.jpeg',scale=6, width=2248, height=450)



In [ ]:
# Subplot 

# create dataframe 
# remove instructions time from beginning and end to make all same length
def trim_lists_to_shortest(input_lists):
    # Find the length of the shortest list
    min_length = min(len(lst) for lst in input_lists)

    # Trim each list to the length of the shortest one
    trimmed_lists = [lst[:min_length] for lst in input_lists]

    return trimmed_lists

eyetracking_data_all_runs_x_trim = trim_lists_to_shortest(eyetracking_data_all_runs_x)
eyetracking_data_all_runs_y_trim = trim_lists_to_shortest(eyetracking_data_all_runs_y)


#data = {"run_01_x": list(eyetracking_data_all_runs_x_trim[0]), "run_01_y": list(eyetracking_data_all_runs_y_trim[0]), "run_02_x": list(eyetracking_data_all_runs_x_trim[1]), "run_02_y": list(eyetracking_data_all_runs_y_trim[1]), "run_03_x": list(eyetracking_data_all_runs_x_trim[2]), "run_03_y": list(eyetracking_data_all_runs_y_trim[2])}
data = {"run_02_x": list(eyetracking_data_all_runs_x_trim[0]), "run_02_y": list(eyetracking_data_all_runs_y_trim[0]), "run_03_x": list(eyetracking_data_all_runs_x_trim[1]), "run_03_y": list(eyetracking_data_all_runs_y_trim[1])}

df = pd.DataFrame(data)
df

In [ ]:
deepmreye_data_all_runs_x_trim = trim_lists_to_shortest(deepmreye_data_all_runs_x)
deepmreye_data_all_runs_y_trim = trim_lists_to_shortest(deepmreye_data_all_runs_y)

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

time_ticks_val = [0, 20000, 40000, 60000, 80000, 100000, 120000, 140000, 160000, 180000, 200000, 220000, 240000, 260000, 280000, 300000, 320000, 340000, 360000, 380000]
time_ticks_text = ['0', '20', "40", "60","80", "100", "120", "140", "160", "180","200", "220", "240", "260", "280","300", "320", "340", "360", "380"]

plot_rows = len(data_eyetrack)
plot_cols = 2

fig = make_subplots(rows=plot_rows, cols=plot_cols,shared_xaxes=True,vertical_spacing=0.05, subplot_titles= ['Hor. Coord. run 1', 'Ver. Coord. run 1', 'Hor. Coord. run 2', 'Ver. Coord. run 2', 'Hor. Coord. run 3', 'Ver. Coord. run 3'])

# Set a common y-axis range
common_y_range = [-22, 22]  # Adjust the range as needed

# add traces
x = 0
for i in range(1, plot_rows + 1):
    for j in range(1, plot_cols + 1):
        trace = go.Scatter(x=df.index, y=df[df.columns[x]].values,
                           name="Eyetracker Gaze Position",
                           mode='lines',
                           showlegend = True,
                           line=dict(color='#0E1C36', width=1.5))

        fig.add_trace(trace, row=i, col=j)

        # Set common Y-axis range
        fig.update_yaxes(range=common_y_range, row=i, col=j)

        x += 1


fig.add_trace(go.Scatter(y= all_runs_expected_x[0],showlegend=True, name='Expected Gaze Position',line=dict(color='#069D6B', width=2)), row = 1, col = 1)
fig.add_trace(go.Scatter(y= all_runs_expected_y[0],showlegend=False, line=dict(color='#069D6B', width=2)), row = 1, col = 2)
fig.add_trace(go.Scatter(y= all_runs_expected_x[1],showlegend=False,line=dict(color='#069D6B', width=2)), row = 2, col = 1)
fig.add_trace(go.Scatter(y= all_runs_expected_y[1],showlegend=False,line=dict(color='#069D6B', width=2)), row = 2, col = 2)
#fig.add_trace(go.Scatter(y= all_runs_expected_x[2],showlegend=False,line=dict(color='#069D6B', width=2)), row = 3, col = 1)
#fig.add_trace(go.Scatter(y= all_runs_expected_y[2],showlegend=False,line=dict(color='#069D6B', width=2)), row = 3, col = 2)


fig.add_vrect(x0="6000", x1="90000", 
            label=dict(
            text="eyes open",
            textposition="top center"),
            fillcolor="#DCDCE5", opacity=0.2, line_width=0)

fig.add_vrect(x0="98000", x1="182000", 
            label=dict(
            text="eyes part. closed",
            textposition="top center"),
            fillcolor="#D8DDEF", opacity=0.2, line_width=0)

fig.add_vrect(x0="190000", x1="273500", 
            label=dict(
            text="no stim.",
            textposition="top center"),
            fillcolor="#A0A4B8", opacity=0.1, line_width=0)

fig.add_vrect(x0="284000", x1="366000", 
            label=dict(
            text="eyes closed",
            textposition="top center"),
            fillcolor="#7293A0", opacity=0.1, line_width=0)

#Format and show fig
fig.update_layout(height=1200, width=3000, template="simple_white", title_text=f"{subject} DeepMReyeClosed Scanner Eyetracker Gaze Position (X,Y) vs. Expected Gaze Position", 
            yaxis1 = dict(title = "<b>Hor. coord. (dva)<b>", title_font=dict(size=12)),
            yaxis2 = dict(title = "<b>Ver. coord. (dva)<b>",title_font=dict(size=12)),
            yaxis3 = dict(title = "<b>Hor. coord. (dva)<b>",title_font=dict(size=12)),
            yaxis4 = dict(title = "<b>Ver. coord. (dva)<b>",title_font=dict(size=12)),
            xaxis5=dict(title='<b>Time (sec)<b>', tickmode = 'array', tickvals = time_ticks_val, ticktext = time_ticks_text),
            yaxis5 = dict(title = "<b>Hor. coord. (dva)<b>", title_font=dict(size=12)),
            xaxis6=dict(title='<b>Time (sec)<b>', tickmode = 'array', tickvals = time_ticks_val, ticktext = time_ticks_text),
            yaxis6 = dict(title = "<b>Ver. coord. (dva)<b>"))

# Update the legend to be not visible for specifc traces
fig.update_traces(showlegend= True, row = 1, col = 1, selector=dict(name= "DeepMReyeClosed Eyetracker Gaze Position"))
fig.update_traces(showlegend=False, row=1, col=2)
fig.update_traces(showlegend=False, row=2, col=1)
fig.update_traces(showlegend=False, row=2, col=2)
#fig.update_traces(showlegend=False, row=3, col=1)
#fig.update_traces(showlegend=False, row=3, col=2)

# Update subplot titles font
fig.update_annotations(font=dict(size=14))


fig.show()
#fig.write_image(f'{subject}_closed_scanner_gazeposition_eyetracker.pdf')


In [ ]:
# Average gaze position 

eyetracking_data_avg_x = (np.array(eyetracking_data_all_runs_x_trim[0]) + np.array(eyetracking_data_all_runs_x_trim[1]) + np.array(eyetracking_data_all_runs_x_trim[2])) / 3
eyetracking_data_avg_y = (np.array(eyetracking_data_all_runs_y_trim[0]) + np.array(eyetracking_data_all_runs_y_trim[1]) + np.array(eyetracking_data_all_runs_y_trim[2])) / 3


In [ ]:
import plotly.graph_objects as go
import numpy as np

N = len(deepmreye_data_all_runs_x[0][::150])
x = deepmreye_data_all_runs_x[0][::150]
y = deepmreye_data_all_runs_y[0][::150]

# Create figure
fig = go.Figure(
    data=[
        go.Scatter(x=x[:1], y=y[:1], mode="lines", line=dict(width=2, color="black")),
    ],
    layout=go.Layout(
        xaxis=dict(range=[-5, 5], autorange=False, zeroline=False),
        yaxis=dict(range=[-5, 5], autorange=False, zeroline=False),
        title_text="DeepMReyeClosed", hovermode="closest",
        updatemenus=[
            dict(
                type="buttons",
                buttons=[dict(label="Play", method="animate", args=[None])]
            )
        ]
    ),
    frames=[
        go.Frame(
            data=[
                go.Scatter(x=x[:k+1], y=y[:k+1], mode="lines", line=dict(width=2, color="black")),
            ],
            name=f"frame{k}"
        )
        for k in range(1, N)
    ]
)

fig.update_layout(
    template='simple_white',
    autosize=False,
    width=800,
    height=800,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100
    )
)

fig.show()


In [ ]:
# Plot Difference in Subplot

import plotly.graph_objects as go
from plotly.subplots import make_subplots



plot_rows = 1
plot_cols = 3

fig = make_subplots(rows=plot_rows, cols=plot_cols,shared_xaxes=False,vertical_spacing=0.05, subplot_titles=["Eyes open", "Eyes part. closed", "No stim."])

# Set a common y-axis range
common_y_range = [-15, 15]  # Adjust the range as needed


fig.add_trace(go.Scatter(x = eyetracking_data_avg_x[:int(trial_durations_all_runs[0][23])] ,y = eyetracking_data_avg_y[:int(trial_durations_all_runs[0][23])], line=dict(color='#4DCBA1', width=2.5),showlegend= False), row = 1, col = 1)
fig.add_trace(go.Scatter(x = eyetracking_data_avg_x[int(trial_durations_all_runs[0][25]):int(trial_durations_all_runs[0][49])] ,y = eyetracking_data_avg_y[int(trial_durations_all_runs[0][25]):int(trial_durations_all_runs[0][49])],line=dict(color='#069D6B', width=2.5),showlegend= False), row = 1, col = 2)
fig.add_trace(go.Scatter(x = eyetracking_data_avg_x[int(trial_durations_all_runs[0][50]):int(trial_durations_all_runs[0][73])],y = eyetracking_data_avg_y[int(trial_durations_all_runs[0][50]):int(trial_durations_all_runs[0][73])],line=dict(color='#3A7E70', width=2.5),showlegend= False), row = 1, col = 3)

fig.add_trace(go.Scatter(x = expected_position_x ,y = expected_position_y, line=dict(color='black', width=1.5),showlegend= False,opacity=0.45), row = 1, col = 1)
fig.add_trace(go.Scatter(x = expected_position_x ,y = expected_position_y,line=dict(color='black', width=1.5,dash='dashdot'),showlegend= False,opacity=0.45), row = 1, col = 2)
fig.add_trace(go.Scatter(x = expected_position_x,y = expected_position_y,line=dict(color='black', width=1.5,dash='dashdot'),showlegend= False,opacity=0.45), row = 1, col = 3)


# Format and show fig
fig.update_layout(height=1200, width=2000, template="simple_white", title_text=f"{subject} Average Gaze Position", 
            yaxis1 = dict(title = "Vert. coord. (dva)", range = [-20,20]),
            xaxis1 = dict(title = "Hor. coord. (dva)", range = [-20,20]),
            yaxis2 = dict(title = "Vert. coord. (dva)", range = [-20,20]),
            xaxis2 = dict(title = "Hor. coord. (dva)", range = [-20,20]),
            yaxis3 = dict(title = "Vert. coord. (dva)", range = [-20,20]),
            xaxis3 = dict(title = "Hor. coord. (dva)", range = [-20,20]))


# Update subplot titles font
fig.update_annotations(font=dict(size=14))

fig.update_layout(
    coloraxis_showscale=False,
    autosize=False,
    width=1500,
    height=600,
    margin=dict(
        l=50,
        r=50,
        b=100,


        t=100,
        pad=4
    ))

fig.update_coloraxes(showscale=False)

fig.show()
fig.write_image(f'{subject}_scanner_gazeposition_closed.pdf')

#colorscale=[[1, 'rgb(255,255,255)'], [0.75, 'rgb(160,164,184)'], [0.5, 'rgb(169,198,184)'], [0.25, 'rgb(216,221,239)'], [0, 'rgb(0,0,0)']]





In [ ]:
import plotly.graph_objects as go



# Define starting indices for slicing
start_indices = [0, 12, 25, 37, 50, 62, 75, 87]

fig = go.Figure()

# Set a common y-axis range
common_y_range = [-15, 15]  # Adjust the range as needed

# Define color for all slices
color = '#4DCBA1'

# Iterate over start indices and plot data slices
for i, start_index in enumerate(start_indices):
    end_index = start_index + 3  
    fig.add_trace(go.Scatter(x=eyetracking_data_avg_x[int(trial_durations_all_runs[0][start_index]):int(trial_durations_all_runs[0][end_index])], y=eyetracking_data_avg_y[int(trial_durations_all_runs[0][start_index]):int(trial_durations_all_runs[0][end_index])], line=dict(color=color, width=2.5), name=f'Slice {i+1}'))

# Add expected position traces
#fig.add_trace(go.Scatter(x=expected_position_x, y=expected_position_y, line=dict(color='black', width=1.5, dash='dashdot'), name='Expected Position', opacity=0.45))

# Format and show fig
fig.update_layout(height=600, width=600, template="simple_white", title_text=f"{subject} Average Gaze Position", 
                  yaxis=dict(title="Vert. coord. (dva)", range=[-15, 15]),
                  xaxis=dict(title="Hor. coord. (dva)", range=[-15, 15]))

# Update annotations font
fig.update_annotations(font=dict(size=14))

fig.update_layout(
    coloraxis_showscale=False,
    autosize=False,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ))

fig.update_coloraxes(showscale=False)

fig.show()
fig.write_image(f'{subject}_scanner_gazeposition_closed.pdf')


In [ ]:
# find triangle down trials 

# Sequence to search
sequence_down = [3, 5, 4]
sequence_left = [3, 4, 1]
sequence_up = [3, 1, 2]
sequence_right  = [3, 2, 5]


sequence = sequence_right

# Find rows where the sequence exists in the 'fixation_position' column
matches = []
for i in range(len(df_run_1) - len(sequence) + 1):
    if df_run_1['fixation_position'].tolist()[i:i+len(sequence)] == sequence:
        matches.append(i)

print(f"Sequence {sequence} found at indices:", matches)

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Define starting indices for slicing
start_indices = matches[:-2]

fig = make_subplots(rows=1, cols=3, shared_xaxes=True, subplot_titles=["Eyes Open", "Eyes Blink", "No Stim."])

# Set a common y-axis range
common_y_range = [-22, 22]  # Adjust the range as needed
common_x_range = [-22, 22]  # Adjust the range as needed

# Define color for all slices
color = '#4DCBA1'

# Iterate over start indices and plot data slices
for i, start_index in enumerate(start_indices):
    end_index = start_index + 3
    x_data = eyetracking_data_avg_x[int(trial_durations_all_runs[0][start_index]):int(trial_durations_all_runs[0][end_index])]
    y_data = eyetracking_data_avg_y[int(trial_durations_all_runs[0][start_index]):int(trial_durations_all_runs[0][end_index])]
    
    if i < 2:
        fig.add_trace(go.Scatter(x=x_data, y=y_data, line=dict(color=color, width=2.5), showlegend=False), row=1, col=1)
        fig.update_yaxes(range=common_y_range, row=1, col=1)
        fig.update_xaxes(range=common_x_range, row=1, col=1)
    elif 2 <= i < 4:
        fig.add_trace(go.Scatter(x=x_data, y=y_data, line=dict(color=color, width=2.5), showlegend=False),  row=1, col=2)
        fig.update_yaxes(range=common_y_range, row=1, col=2)
        fig.update_xaxes(range=common_x_range, row=1, col=2)
    else:
        fig.add_trace(go.Scatter(x=x_data, y=y_data, line=dict(color=color, width=2.5), showlegend=False), row=1, col=3)
        fig.update_yaxes(range=common_y_range, row=1, col=3)
        fig.update_xaxes(range=common_x_range, row=1, col=3)

# Add expected position traces
# fig.add_trace(go.Scatter(x=expected_position_x, y=expected_position_y, line=dict(color='black', width=1.5, dash='dashdot'), name='Expected Position', opacity=0.45))

# Format and show fig
fig.update_layout(height=600, width=2000, template="simple_white", title_text=f"{subject} Average Gaze Position Triangle Right", 
            yaxis1 = dict(title = "Vert. coord. (dva)"),
            xaxis1 = dict(title = "Hor. coord. (dva)"),
            yaxis2 = dict(title = "Vert. coord. (dva)"),
            xaxis2 = dict(title = "Hor. coord. (dva)"),
            yaxis3 = dict(title = "Vert. coord. (dva)"),
            xaxis3 = dict(title = "Hor. coord. (dva)"))

# Update annotations font
fig.update_annotations(font=dict(size=14))

fig.update_layout(
    coloraxis_showscale=False,
    autosize=False,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ))

fig.update_coloraxes(showscale=False)

fig.show()
fig.write_image(f'{subject}_scanner_gazeposition_closed.pdf')


In [ ]:
def pointwise_difference(array1, array2):
    """
    Calculate the pointwise difference between two arrays.

    Parameters:
    - array1: First input array
    - array2: Second input array

    Returns:
    - result: Array containing the pointwise differences
    """
    

    result = [a - b for a, b in zip(array1, array2)]
    return result

In [ ]:
# difference of eye tracking and deepmreye

difference_x_run_1 = pointwise_difference(eyetracking_data_all_runs_x_trim[0],deepmreye_data_all_runs_x_trim[0])
difference_x_run_2 = pointwise_difference(eyetracking_data_all_runs_x_trim[1],deepmreye_data_all_runs_x_trim[1])
difference_x_run_3 = pointwise_difference(eyetracking_data_all_runs_x_trim[2],deepmreye_data_all_runs_x_trim[2])

In [ ]:
# difference of eye tracking and deepmreye

difference_y_run_1 = pointwise_difference(eyetracking_data_all_runs_y_trim[0],deepmreye_data_all_runs_y_trim[0])
difference_y_run_2 = pointwise_difference(eyetracking_data_all_runs_y_trim[1],deepmreye_data_all_runs_y_trim[1])
difference_y_run_3 = pointwise_difference(eyetracking_data_all_runs_y_trim[2],deepmreye_data_all_runs_y_trim[2])

In [ ]:
difference_x_avg = (np.array(difference_x_run_1) + np.array(difference_x_run_2) + np.array(difference_x_run_3)) / 3
difference_y_avg = (np.array(difference_y_run_1) + np.array(difference_y_run_2) + np.array(difference_y_run_3)) / 3

In [ ]:
# Plot Averge Difference Histogram in Subplot

import plotly.graph_objects as go
from plotly.subplots import make_subplots



plot_rows = 1
plot_cols = 3

fig = make_subplots(rows=plot_rows, cols=plot_cols,shared_xaxes=False, vertical_spacing=0.05, subplot_titles=["Eyes Open", "Blink", "No Stim."])

# Set a common y-axis range
common_y_range = [-15, 15]  # Adjust the range as needed


fig.add_trace(go.Histogram2d(x = difference_x_avg[:int(trial_durations_all_runs[0][23])],y = difference_y_avg[:int(trial_durations_all_runs[0][23])],  name = "Fixation difference", colorscale='gray', reversescale= True, showscale = False), row = 1, col = 1)
fig.add_trace(go.Histogram2d(x = difference_x_avg[int(trial_durations_all_runs[0][25]):int(trial_durations_all_runs[0][49])],y = difference_y_avg[int(trial_durations_all_runs[0][25]):int(trial_durations_all_runs[0][49])], colorscale= 'gray', reversescale= True,  name = "Pursuit difference", showscale = False), row = 1, col = 2)
fig.add_trace(go.Histogram2d(x = difference_x_avg[int(trial_durations_all_runs[0][50]):int(trial_durations_all_runs[0][73])],y = difference_y_avg[int(trial_durations_all_runs[0][25]):int(trial_durations_all_runs[0][49])], name = "All difference",colorscale= 'gray', reversescale= True,showscale = True), row = 1, col = 3)




# Format and show fig
fig.update_layout(height=1200, width=2000, template="simple_white", title_text=f"{subject}: Average histograms of pretrained deepmreye positions relative to expected positions (0, 0)",  #difference? 
            yaxis1 = dict(title = "<b>Ver. coord. (dva)<b>", range = [-15,15], title_font=dict(size=10)),
            xaxis1 = dict(title = "<b>Hor. coord. (dva)<b>", range = [-15,15], title_font=dict(size=10)),
            yaxis2 = dict(title = "<b>Ver. coord. (dva)<b>", range = [-15,15], title_font=dict(size=10)),
            xaxis2 = dict(title = "<b>Hor. coord. (dva)<b>", range = [-15,15], title_font=dict(size=10)),
            yaxis3 = dict(title = "<b>Ver. coord. (dva)<b>", range = [-15,15], title_font=dict(size=10)),
            xaxis3 = dict(title = "<b>Hor. coord. (dva)<b>", range = [-15,15], title_font=dict(size=10)))


# Update subplot titles font
fig.update_annotations(font=dict(size=14))

fig.update_layout(
    coloraxis_showscale=False,
    autosize=False,
    width=1600,
    height=600,
    margin=dict(
        l=50,
        r=50,
        b=100,


        t=100,
        pad=4
    ))

fig.update_coloraxes(showscale=False)

fig.show()
#fig.write_image(f'{subject}_scanner_eyetracking_relative_gaze_avg.pdf')

#colorscale=[[1, 'rgb(255,255,255)'], [0.75, 'rgb(160,164,184)'], [0.5, 'rgb(169,198,184)'], [0.25, 'rgb(216,221,239)'], [0, 'rgb(0,0,0)']]



